In [30]:
import os
%pwd

'/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project'

In [31]:
os.chdir('/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project')
%pwd

'/Users/rajusubba/Documents/End-to-End MLOps/customer-churn-project'

In [32]:
from dataclasses import dataclass
from pathlib import Path    

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str

    model_type: str
    C: float
    max_iter: int

In [33]:
from src.customerchurn.constants import *
from src.customerchurn.utils.common import read_yaml, create_directories

In [39]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH,
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_trainer

        create_directories([config.root_dir])

        lr_params = params.logistic_regression

        return ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            target_column=self.schema.target_column,

            model_type=params.model_type,
            C=float(lr_params.C),
            max_iter=int(lr_params.max_iter),
    )

In [40]:
import pandas as pd
import os
from src.customerchurn.logging.logger import logger
from sklearn.linear_model import LogisticRegression
import joblib



In [41]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]   # ✅ Series

        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]

        model = LogisticRegression(C=self.config.C, max_iter=self.config.max_iter)
        model.fit(X_train, y_train)

        os.makedirs(self.config.root_dir, exist_ok=True)
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(model, model_path)

        logger.info(f"Saved model to: {model_path}")
        return model_path
        

In [42]:
try:
    config_manager = ConfigurationManager()
    trainer_config = config_manager.get_model_trainer_config()
    
    model_trainer = ModelTrainer(config=trainer_config)
    model_path = model_trainer.train_model()
    logger.info("Model training completed successfully.")
except Exception as e:
    logger.error(f"Error during model training: {e}")
    raise e

[2026-02-05 12:11:20,752: INFO: common: YAML file config/config.yaml loaded successfully.]
[2026-02-05 12:11:20,754: INFO: common: YAML file params.yaml loaded successfully.]
[2026-02-05 12:11:20,756: INFO: common: YAML file schema.yaml loaded successfully.]
[2026-02-05 12:11:20,756: INFO: common: Directory created at: artifacts]
[2026-02-05 12:11:20,757: INFO: common: Directory created at: artifacts/model_trainer]
[2026-02-05 12:11:22,368: INFO: 2019259287: Saved model to: artifacts/model_trainer/model.joblib]
[2026-02-05 12:11:22,369: INFO: 1266605626: Model training completed successfully.]
